## Цель иследования
 - выявить частоту (периоды) повторяемости наблюдаемой характеристики 'rang_namber'
 - создать модель для ML позволяющую предиктить периоды на новых данных 
 - предположить факторы влияющие на чатсоту повторяемости
 - https://github.com/julianandrews/flopferret/blob/master/flopferret/board_texture.py

In [588]:
import re
import pandas as pd
from datetime import datetime
import eval7, pprint
from poker.room.pokerstars import PokerStarsHandHistory
from poker.hand import Hand, Combo
import numpy as np
from phevaluator import evaluate_cards
import matplotlib.pyplot as plt
plt.style.use('dark_background') #стиль графиков

In [589]:
# Создали словарь
data_pd = {'TypeOfRoom': [], 'HandId': [], 'Tournament': [], 'Level': [], 'Date': [], 'Hour': [],
           'TimeZoneAbb': [], 'Table': [], 'NoOfPlayers': [], 'bain': [], 'HandHiro': [], 'Board': []}

# Открываем файл
with open('dim1000.txt', 'r') as f: #  Poker_\SNG dimetruk 1-54133.txt    Poker_\SNG dimetruk 1-56194.txt
    data = f.read()
    
# Ищем нужные данные
result = re.findall('(\w+) Hand #(\d*): Tournament #(\d*),(.*) Level (.*) - (\d{4}/\d+/\d+) (\d+\D\d+\D\d+) (\w*)', data)
Tables = re.findall('Table \'.*\' (.*) Seat .*\n', data)
bains = re.findall('(\D\d+\.\d+\+\D\d+\.\d+)', data)
HandHiros = re.findall("Dealt to dimetruk \D(.. ..)\D", data)
Boards = re.findall("Board \[(.*)\]", data)


# Считаем сколко фишек
s = re.findall('Seat (\d).*chips', data)
s = [int(x) for x in s]
seats = []
length = 1
for i in range(len(s)-1):
    if s[i+1] < s[i]:
        seats.append(length)
        length = 1
    else:
        length += 1
seats.append(length)


# ЗАполнЯем словарь данными
for r, table, bain, seat, handHiro, board in zip(result, Tables, bains, seats, HandHiros, Boards):
    data_pd['TypeOfRoom'].append(r[0])
    data_pd['HandId'].append(r[1])
    data_pd['Tournament'].append(r[2])
    # data_pd['info'].append(r[3])
    data_pd['Level'].append(r[4])
    data_pd['Date'].append(r[5])
    data_pd['Hour'].append(r[6])
    data_pd['TimeZoneAbb'].append(r[7])
    data_pd['Table'].append(table) #table
    data_pd['NoOfPlayers'].append(seat)
    data_pd['bain'].append(bain)
    data_pd['HandHiro'].append(handHiro)
    data_pd['Board'].append(board)

# Создаём ДатаФрейм
dataf = pd.DataFrame(data_pd)
dataf.head()

,TypeOfRoom,HandId,Tournament,Level,Date,Hour,TimeZoneAbb,Table,NoOfPlayers,bain,HandHiro,Board
0,PokerStars,226484810600,3187437334,IV (40/80),2021/05/05,19:56:10,MSK,6-max,2,$13.79+$1.21,Jh Td,8h 8c 7c 3h 6d
1,PokerStars,226484828418,3187437334,V (60/120),2021/05/05,19:56:57,MSK,6-max,2,$13.79+$1.21,Ts 3s,2s 6s As Jc Th
2,PokerStars,226484843891,3187437334,V (60/120),2021/05/05,19:57:39,MSK,6-max,2,$13.79+$1.21,4s 5c,Ks Qh 7h Tc 4c
3,PokerStars,226484859594,3187437334,V (60/120),2021/05/05,19:58:21,MSK,6-max,2,$13.79+$1.21,9c Th,7s 4d Qd
4,PokerStars,226484865470,3187437334,V (60/120),2021/05/05,19:58:37,MSK,6-max,2,$13.79+$1.21,7h 9d,2h Qd 5d Ts 3s


## Преобразуем строковые данные

In [590]:
date = data_pd['Date']
time = data_pd['Hour']

dataf["Date"] = pd.to_datetime(data_pd['Date'])
dataf["Time"] = pd.to_timedelta(data_pd['Hour'])

dataf["DateTime"] = dataf["Date"] + dataf["Time"]
dataf["DateTime"]

# filter by single day
dataf_m = dataf[dataf['DateTime'].dt.strftime('%d') == '06']

dataf.Tournament = dataf.Tournament.astype('int64')
dataf = dataf.astype({'HandId': np.int64, 'Tournament': np.int64})

In [591]:
# dataf["Date"]

In [592]:
# дневной период  https://askdev.ru/q/kak-gruppirovat-i-podschityvat-stroki-po-mesyacam-i-godam-s-pomoschyu-pand-511317/
# df_day = dataf['DateTime'].groupby(dataf.DateTime.dt.to_period("D")).agg('count')
# df_day

In [593]:
# # Получим разницу между датами в днях    https://habr.com/ru/company/wunderfund/blog/677068/
# df["days_to_checkin"] = \
# (df["checkin_date"] - df["created_at"]).dt.days
# # вывод объекта DataFrame
# df

In [594]:
# nov_mar_series = pd.Series(pd.date_range("2013-11-15", "2014-03-15"))
# #create timestamp without year
# nov_mar_no_year = nov_mar_series.map(lambda x: x.strftime("%m-%d"))
# nov_mar_no_year

In [595]:
# #add a yearless timestamp to the dataframe
# df["no_year"] = df['nov_mar_series'].map(lambda x: x.strftime("%m-%d"))
# no_year_mask = df['no_year'].isin(nov_mar_no_year)
# df[no_year_mask]

## Фильтруем турниры 

In [596]:
data_filter = dataf[['Date', 'HandHiro', 'Board','HandId']]
data_filter.head()

,Date,HandHiro,Board,HandId
0,2021-05-05,Jh Td,8h 8c 7c 3h 6d,226484810600
1,2021-05-05,Ts 3s,2s 6s As Jc Th,226484828418
2,2021-05-05,4s 5c,Ks Qh 7h Tc 4c,226484843891
3,2021-05-05,9c Th,7s 4d Qd,226484859594
4,2021-05-05,7h 9d,2h Qd 5d Ts 3s,226484865470


In [597]:
# data_filter.groupby(['Tournament']).agg({'HandId': "count"}).reset_index()

In [598]:
#  Фильтруйте value_counts с помощью isin  https://softhints.com/pandas-how-to-filter-results-of-value_counts/
# data_filter[data_filter['Tournament'].isin(data_filter['Tournament'].value_counts()[data_filter['Tournament'].value_counts()>30].index)].Tournament


In [599]:
#  Используйте group by и lambda для имитации фильтра для value_counts()
# df = data_filter.groupby('Tournament').filter(lambda x: len(x) > (data_filter['Tournament'].value_counts().mean()))
# df

In [600]:
#df.Tournament.value_counts()

In [601]:
# Prepare Data
# df_plot = df_day.groupby('DateTime').size()
# df_plot=dataf['DateTime'].groupby(dataf.DateTime.dt.to_period("D")).agg('count')

# # Make the plot with pandas
# df_plot.plot(kind='pie', subplots=True,autopct="%.2f", figsize=(8, 8))
# plt.title("Колличество комбинаций/день")
# plt.ylabel("")
# plt.show()

In [602]:
# объеденяем руку и борд, выводим название комбинации

# df['hand'] = df['HandHiro'] + " " + df['Board']
# # hand = df['hand'].values[7].split()
# # print(hand)

# hand_ev = [eval7.Card(s) for s in (df['hand'].values[7].split())]
# rang = eval7.evaluate(hand_ev)   # ранг комбинации
# # print(rang)

# eval7s = eval7.handtype(rang)  #  название комбинации

# df['rang'] = eval7.evaluate(hand_ev)
# df['eval7s'] =  eval7.handtype(rang)

In [603]:
# def new_type(row):
#     if row.traffic_source.lower() in ('yandex', 'google'):
#         return 'organic'
#     elif row.traffic_source.lower() in ('paid', 'email'):
#         if row.region.lower() == 'russia':
#             return 'ad'
#         else:
#             return 'other'

In [604]:
# log['source_type'] = log.apply(new_type, axis = 1)
# log.head(12)

In [605]:
# log.loc[log.traffic_source.isin(['yandex', 'google']), 'traffic_type'] = 'organic' 

In [606]:
df = dataf

df['hand_rang'] = df['HandHiro'] + " " + df['Board']
df['hand_rang']

print(type(df['hand_rang']), type(df['hand_rang'].values[7].split()))
df['hand_rang'].values[0].split()

<class 'pandas.core.series.Series'> <class 'list'>


['Jh', 'Td', '8h', '8c', '7c', '3h', '6d']

In [607]:
# df['hand_rang'].to_list()
# df['hand_rang'].tolist()
# np.array(df['hand_rang'].tolist())
# Ls = list(df['hand_rang'])
ls = df['hand_rang'].str.split()
ls

0      [Jh, Td, 8h, 8c, 7c, 3h, 6d]
1      [Ts, 3s, 2s, 6s, As, Jc, Th]
2      [4s, 5c, Ks, Qh, 7h, Tc, 4c]
3              [9c, Th, 7s, 4d, Qd]
4      [7h, 9d, 2h, Qd, 5d, Ts, 3s]
                   ...             
995        [5s, 6s, 3d, 7d, 2s, 2h]
996    [8d, 9c, 5s, Th, 4c, 7s, Js]
997            [Td, 6h, 9c, Kc, Jc]
998    [Ts, Kh, 4c, 9h, 7c, 5d, 8s]
999    [Tc, Th, Qd, 2d, Ah, 6d, 3d]
Name: hand_rang, Length: 1000, dtype: object

In [608]:
# напишем функцию получения ранг комбинации

def evaluate_(hand):
    """
    На вход получает карты, на выход 
    число ранг по eval7
    """
    hand_ev = [eval7.Card(s) for s in (hand)]
    return eval7.evaluate(hand_ev)

a = ('Jh', 'Td', '8h', '8c', '7c', '3h', '6d')
evaluate_(a)

17209424

In [609]:
# напишем функцию получения названия ранга

def handtype_(rang):
    """
    На вход получает ранг, на выход 
    название комбинации
    """
    return eval7.handtype(rang)

handtype_(17209424)

'Pair'

In [610]:
# Применим функцию к столбцу hand  и запишем в новый столбец rang_namber

df['rang_namber'] = df['hand_rang'].str.split().apply(evaluate_)
df['handtypes'] = df['rang_namber'].apply(handtype_)
df = df[['Date', 'handtypes', 'rang_namber','HandId']]
df.head()
# df['filter'] = df.loc[df.handtypes.str.contains('Two Pair', 'Trips', 'Straight', 'Full House', 'Quads', 'Straight Flush'), 'rang_namber']

,Date,handtypes,rang_namber,HandId
0,2021-05-05,Pair,17209424,226484810600
1,2021-05-05,Flush,84706320,226484828418
2,2021-05-05,Pair,16956032,226484843891
3,2021-05-05,High Card,690002,226484859594
4,2021-05-05,High Card,690003,226484865470


In [611]:
# df.sort_values(by=['handtypes', 'rang_namber'])

In [612]:
# df.groupby('handtypes')['rang_namber'].value_counts()

### Создадим сводную таблицу, что-бы проверить наличие аномалий

In [613]:
# df_mylty = pd.MultiIndex.from_frame(df)
# df_mylty

In [614]:
# df.reindex(df_mylty)    

In [615]:
df_pivot = pd.pivot_table(df, index=[df.Date.dt.to_period("M"), "handtypes"],
                          values=["rang_namber"],
                          aggfunc= len)
df_pivot

rang_namber
Date    handtypes              
2021-05 Flush                16
        Full House           21
        High Card           232
        Pair                328
        Straight             24
        Trips                23
        Two Pair            127
2021-06 Flush                 1
        High Card             3
        Pair                  5
2021-07 Flush                 1
        Full House            4
        High Card            64
        Pair                 90
        Quads                 1
        Straight              6
        Trips                 6
        Two Pair             27
2021-08 Full House            1
        High Card             6
        Pair                 10
        Straight              1
        Trips                 1
        Two Pair              2

In [616]:
# колличесвто комбинаций по месяцам
df.pivot_table(['handtypes'], columns=df.Date.dt.to_period("W"), aggfunc=len)

Date,2021-05-03/2021-05-09,2021-05-10/2021-05-16,2021-05-24/2021-05-30,2021-05-31/2021-06-06,2021-07-05/2021-07-11,2021-07-12/2021-07-18,2021-08-02/2021-08-08
handtypes,573,33,165,9,124,75,21


In [617]:
# сводная таблица
df.pivot_table(index=df.Date.dt.to_period("W"),                # задаём период в днях D или месяцах M 
               columns='handtypes' ,                           #подсчёт по этим значениям
               aggfunc={'rang_namber':len})                   # задаём столбци и вычисление - агрегатную функцию

rang_namber                                             \
handtypes                   Flush Full House High Card   Pair Quads Straight   
Date                                                                           
2021-05-03/2021-05-09        13.0       14.0     174.0  244.0   NaN     15.0   
2021-05-10/2021-05-16         NaN        2.0      11.0   16.0   NaN      2.0   
2021-05-24/2021-05-30         3.0        5.0      47.0   68.0   NaN      7.0   
2021-05-31/2021-06-06         1.0        NaN       3.0    5.0   NaN      NaN   
2021-07-05/2021-07-11         NaN        2.0      43.0   51.0   1.0      5.0   
2021-07-12/2021-07-18         1.0        2.0      21.0   39.0   NaN      1.0   
2021-08-02/2021-08-08         NaN        1.0       6.0   10.0   NaN      1.0   

                                      
handtypes             Trips Two Pair  
Date                                  
2021-05-03/2021-05-09  14.0     99.0  
2021-05-10/2021-05-16   1.0      1.0  
2021-05-24/2021-05-30   8.0     27.0  
2021-05-31/2021-06-06   NaN      NaN  
2021-07-05/2021-07-11   4.0     18.0  
2021-07-12/2021-07-18   2.0      9.0  
2021-08-02/2021-08-08   1.0      2.0

In [623]:
df.plot(kind='line')

<AxesSubplot:>

OverflowError: int too big to convert

<Figure size 432x288 with 1 Axes>

In [622]:
df.pivot_table(df.('region','area','distributor','salesrep'),
  aggfunc=np.sum, margins=True)

SyntaxError: invalid syntax (Temp/ipykernel_9720/2095209704.py, line 1)

In [ ]:
# df_mylty = pd.MultiIndex.from_frame(df_pivot)
# df_mylty

In [ ]:
# df_raw = df.groupby([df.Date.dt.to_period("M"), "handtypes"])['rang_namber'].count()
# df_raw

In [ ]:
# дневной период  https://askdev.ru/q/kak-gruppirovat-i-podschityvat-stroki-po-mesyacam-i-godam-s-pomoschyu-pand-511317/
df_day = dataf['DateTime'].groupby(dataf.DateTime.dt.to_period("D")).agg('count')
df_day

In [ ]:
# явно определив столбцы, которые нам нужны,
pd.pivot_table(df,
               index=[ "handtypes","Date"],
               values=["rang_namber"],
               aggfunc= len)  # aggfunc=[np.mean, len])

In [ ]:
df['Date'].value_counts()

In [ ]:
# df['Time'].value_counts()

In [ ]:
# Prepare Data
df_plot = df.groupby('handtypes').size()

# Make the plot with pandas
df_plot.plot(kind='area', subplots=True, figsize=(8, 8))
plt.title("Колличество комбинаций")
plt.ylabel("")
plt.show()

In [ ]:
# df['DateTime'] = df['DateTime'][(df['DateTime'] > '2018-10-01 20:00:00') & (df['DateTime'] < '2022-10-02 00:00:00')]
# df['DateTime']


In [ ]:
# отфильтруем комбинации "старшая карта" и "пара"
df['no_par'] = df['rang_namber'][df['rang_namber'] > 50000000] #[((df['rang_namber'] > 50000000) & (df['rang_namber'] < 60000000))]
df = df.dropna()
df[['Time', 'no_par', 'handtypes']].head(10)
# df['filter_Two Pair'] = df['handtypes'][df['handtypes'] == 'Two Pair']
# df['filter_Two Pair']

In [ ]:
df.isnull().sum()

In [ ]:
# преобразуем время во временной ряд


In [ ]:


idx = df['Time']
 # datetime.timedelta(seconds=24*60*60).total_seconds()

# second = df['DateTime'].dt.total_seconds()
# second

# idx = pd.date_range(df['DateTime'], periods=5, freq="H")
# idx
tim = pd.Series(range(len(idx)), index=idx)

# ts.resample("2H").mean()
tim

In [ ]:
# df = df.set_index("Time")
# df

In [ ]:

import matplotlib.pyplot as plt
 
# Using a inbuilt style to change
# the look and feel of the plot
plt.style.use("classic")
 
# setting figure size to 12, 10
plt.figure(figsize=(12, 10))
 
# Labelling the axes and setting
# a title
plt.xlabel("Time")
plt.ylabel("Values")
plt.title("Sample Time Series Plot")
 
# plotting the "A" column alone
plt.plot(df["handtypes"])

In [ ]:
df.plot(subplots=True, figsize=(12, 15))

In [ ]:
# # # idx = pd.date_range('2019-01-01', periods=48, freq='H')
# # x = pd.Series(range(len(idx)), index=idx)
# y = tim
# # add 'hour' and 'day' columns in the dataframe
# d = {'x': x, 'y': y, 'hour': idx.hour, 'day': idx.day}
# df = pd.DataFrame(data=d)
# # use 'hour' as x axis to plot, and 'day' as marker color
# df.plot.scatter(x='hour', y='y', c='day', colormap='rainbow')
# plt.show()

In [ ]:
data.dtypes

In [ ]:
# установитьdatetimeОбъект в качестве индекса кадра данных
data = data.set_index('DateTime')
data.head()

In [ ]:

# data.plot(grid=True)

data['filter'].plot(kind='bar')

# import matplotlib.pyplot as plt
# from matplotlib.dates import DateFormatter
# # Группировка за месяц
# df_month = data.resample("M").mean()

# fig, ax = plt.subplots(figsize=(10, 6))
# ax.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
# ax.bar(
#     df_month['2018':].index, 
#     df_month.loc['2016':, "filter"], 
#     width=25, align='center')

In [ ]:

data.plot(kind='scatter', 
        x='DateTime', 
        y='filter', 
        title='Связь между жёсткостью воды и средней годовой смертностью')